In [ ]:
# default_exp funcs.train_funcs

In [ ]:
%load_ext autoreload
%autoreload 2

# Training loop simulator learning

In [ ]:
#export
from decode_fish.imports import *
from decode_fish.funcs.evaluation import *
from decode_fish.funcs.file_io import *
from decode_fish.funcs.emitter_io import *
from decode_fish.funcs.utils import *
from decode_fish.funcs.dataset import *
from decode_fish.funcs.output_trafo import *
from decode_fish.funcs.plotting import *
import torch.nn.functional as F
from torch import distributions as D
from torch.optim import AdamW
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader
from decode_fish.engine.microscope import Microscope
from decode_fish.engine.model import UnetDecodeNoBn
import shutil
from torch.utils.tensorboard import SummaryWriter
from decode_fish.engine.point_process import PointProcessUniform
from decode_fish.engine.gmm_loss import PointProcessGaussian
import wandb
from madgrad import MADGRAD
# from decode_fish.funcs.visualization vimport get_simulation_statistics

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [ ]:
#export
def eval_logger(pred_df, target_df, iteration, data_str='Sim. '):
    
    perf_dict,_,shift = matching(target_df, pred_df, print_res=False)
    if 'Inp' in data_str:
        pred_corr = shift_df(pred_df, shift)
        perf_dict, _, _ = matching(target_df, pred_corr, print_res=False)

    wandb.log({data_str +'Metrics/eff_3d': perf_dict['eff_3d']}, step=iteration)
    wandb.log({data_str +'Metrics/jaccard': perf_dict['jaccard']}, step=iteration)
    wandb.log({data_str +'Metrics/rmse_vol': perf_dict['rmse_vol']}, step=iteration)

    wandb.log({data_str +'Metrics/precision': perf_dict['precision']}, step=iteration)
    wandb.log({data_str +'Metrics/recall': perf_dict['recall']}, step=iteration)
    wandb.log({data_str +'Metrics/rmse_x': perf_dict['rmse_x']}, step=iteration)
    wandb.log({data_str +'Metrics/rmse_y': perf_dict['rmse_y']}, step=iteration)
    wandb.log({data_str +'Metrics/rmse_z': perf_dict['rmse_z']}, step=iteration)   
    
def load_from_eval_dict(eval_dict):
    
    eval_img = load_tiff_image(sorted(glob.glob(eval_dict['image_path']))[eval_dict['img_ind']])
    eval_img = eval_img[eval_dict['crop_sl']]
    eval_df = None
    eval_psf = None
    if eval_dict['txt_path'] is not None:
        txt_path = sorted(glob.glob(eval_dict['txt_path']))[eval_dict['img_ind']]
        eval_df = simfish_to_df(txt_path)
        eval_df = crop_df(eval_df, eval_dict['crop_sl'], px_size_zyx=eval_dict['px_size_zyx'])

    if eval_dict['psf_path'] is not None:
        eval_psf = load_tiff_image(eval_dict['psf_path'])
        
    return eval_img, eval_df, eval_psf

def save_train_state(save_dir, model, microscope, optim_net, psf, optim_psf):
    
        torch.save({'state_dict':model.state_dict(), 'scaling':[model.unet.inp_scale, model.unet.inp_offset]}, save_dir/'model.pkl')
        torch.save(microscope.state_dict(), save_dir/'microscope.pkl')
        torch.save(optim_net.state_dict(), save_dir/'opt_net.pkl')
        torch.save(psf.state_dict(), save_dir/'psf.pkl' )
        torch.save(optim_psf.state_dict(), save_dir/'opt_psf.pkl')

In [ ]:
#export
def train(cfg,
          model, 
          dl, 
          optim_net, 
          optim_psf, 
          optim_mic,
          sched_net, 
          sched_psf, 
          sched_mic,
          microscope,
          psf,
          post_proc,
          eval_dict=None):
    
    """
    Training loop for autoencoder learning. Alternates between a simulator training step to train the inference network
    and an autoencoder step to train the PSF (and microscope) parameters.
    
    Args:
        model (torch.nn.Module): DECODE 3D UNet.
        num_iter (int): Number of training iterations for pure sl learning(batches).
        num_iter (int): Total number of training iterations (batches).
        dl  (torch.utils.data.dataloader.DataLoader): Dataloader that returns a random sub volume from the real volume, an estiamted emitter density and background.
        optim_net  (torch.optim.Optimizer): Optimizer for the network parameters.
        optim_psf  (torch.optim.Optimizer): Optimizer for the PSF parameters.
        sched_net  (torch.optim.lr_scheduler): LR scheduler for the network parameters.
        sched_psf  (torch.optim.lr_scheduler): LR scheduler for the PSF parameters.
        min_int  (float): Minimal fraction of the max intensity used when sampling emitters. 
        microscope (torch.nn.Module): Microscope class that transforms emitter locations into simulated images.
        log_interval  (int): Number of iterations between performance evaluations.
        save_dir  (str, PosixPath): Output path where the trained model is stored.
        log_dir  (str, PosixPath, optional): Output path where log files for Tensorboard are stored.
        psf (torch.nn.Module): Parametric PSF.
        bl_loss_scale  (float): The background loss gets scaled by this factor when added to the GMM loss.
        grad_clip  (float): Gradient clipping threshold. 
        eval_dict  (dict, optional): Dictionary with evaluation parameters
        
    """ 
    
    save_dir = Path(cfg.output.save_dir)
    
    if eval_dict is not None:
        eval_img, eval_df, eval_psf = load_from_eval_dict(eval_dict)

    model.cuda().train()
    torch.save(psf.state_dict(), str(save_dir) + '/psf_init.pkl' )

    for batch_idx in range(cfg.training.num_iters):

        x, local_rate, background = next(iter(dl))
        
        optim_net.zero_grad()
        
        sim_vars = PointProcessUniform(local_rate, int_conc=model.int_dist.int_conc.detach(), int_rate=model.int_dist.int_rate.detach(), int_loc=model.int_dist.int_loc.detach(), sim_iters=5).sample()  
        # sim_vars = locs_sl, x_os_sl, y_os_sl, z_os_sl, ints_sl, output_shape
        xsim = microscope(*sim_vars)
        xsim_noise = microscope.noise(xsim, background).sample()

        out_sim = model(xsim_noise)

        count_prob, spatial_prob = PointProcessGaussian(**out_sim).log_prob(*sim_vars[:5])
        gmm_loss = -(spatial_prob + cfg.training.net.cnt_loss_scale*count_prob).mean()
        
        background_loss = F.mse_loss(out_sim['background'], background) * cfg.training.net.bl_loss_scale

        loss = gmm_loss + background_loss
        
        # Update network parameters
        loss.backward()
        
        if cfg.training.net.grad_clip: torch.nn.utils.clip_grad_norm_(model.unet.parameters(), max_norm=cfg.training.net.grad_clip, norm_type=2)

        optim_net.step()
        if sched_net:
            sched_net.step()
            
        if batch_idx > cfg.training.start_micro:
            
            out_inp = model(x)
            proc_out_inp = post_proc(out_inp, ret='micro') # locations, x_os_3d, y_os_3d, z_os_3d, ints_3d, output_shape, comb_sig
            
            if batch_idx > cfg.training.start_psf:
                
                optim_psf.zero_grad()

                # Get autoencoder loss
                ae_img = microscope(*proc_out_inp[:6])
                log_p_x_given_z = -microscope.noise(ae_img,out_inp['background']).log_prob(x).mean()
                if cfg.training.psf.norm_reg:
#                     log_p_x_given_z += cfg.training.psf.norm_reg * (psf.sum_loss() + psf.com_loss() + psf.clip_loss())
                    log_p_x_given_z += cfg.training.psf.norm_reg * (psf.com_loss())

                log_p_x_given_z.backward()
                if cfg.training.psf.grad_clip:
                    torch.nn.utils.clip_grad_norm_(optim_psf.param_groups[0]['params'], max_norm=cfg.training.psf.grad_clip, norm_type=2)
                
                optim_psf.step()
                if sched_psf:
                    sched_psf.step()
            
            if len(proc_out_inp[4]):
                
                optim_mic.zero_grad()
                if cfg.training.isi_filt == True:
                    good_ints = proc_out_inp[4][proc_out_inp[7] == 1.]
                else:
                    good_ints = proc_out_inp[4]
                #   good_ints = proc_out_inp[4][proc_out_inp[6] < torch.quantile(proc_out_inp[6], cfg.training.micro.int_quantile)]

                gamma_int = D.Gamma(model.int_dist.int_conc, model.int_dist.int_rate)
                loc_trafo = [D.AffineTransform(loc=model.int_dist.int_loc, scale=1, event_dim=1)]
                int_loss = -D.TransformedDistribution(gamma_int, loc_trafo).log_prob(good_ints.detach()).mean()
                
#                 print('Pars:', model.int_dist.int_conc.detach(), model.int_dist.int_rate.detach(),model.int_dist.int_loc.detach())
#                 print('Ints:', good_ints.detach())
#                 print('Loss:', int_loss)
                
                int_loss.backward()
                optim_mic.step()
                if sched_mic:
                    sched_mic.step()

        # Logging
        if batch_idx % 10 == 0:
            wandb.log({'SL Losses/gmm_loss': gmm_loss.detach().cpu()}, step=batch_idx)
            wandb.log({'SL Losses/count_loss': (-count_prob.mean()).detach().cpu()}, step=batch_idx)
            wandb.log({'SL Losses/bg_loss': background_loss.detach().cpu()}, step=batch_idx)
            wandb.log({'AE Losses/int_mu': model.int_dist.int_conc.item()/model.int_dist.int_rate.item() + model.int_dist.int_loc.item()}, step=batch_idx)
            wandb.log({'AE Losses/int_rate': model.int_dist.int_rate.item()}, step=batch_idx)
            wandb.log({'AE Losses/int_loc': model.int_dist.int_loc.item()}, step=batch_idx)
            wandb.log({'AE Losses/theta': microscope.noise.theta.item()}, step=batch_idx)

            if batch_idx > cfg.training.start_psf: 
                wandb.log({'AE Losses/p_x_given_z': log_p_x_given_z.detach().cpu()}, step=batch_idx)
                wandb.log({'AE Losses/RMSE(rec)': torch.sqrt(((x-(ae_img+out_inp['background']))**2).mean()).detach().cpu()}, step=batch_idx)
                wandb.log({'AE Losses/sum(psf)': F.relu(psf.psf_volume/psf.psf_volume.max())[0].sum().detach().cpu()}, step=batch_idx)
#                     wandb.log({'AE Losses/theta': microscope.theta.item()}, step=batch_idx)
        
        if batch_idx % cfg.output.log_interval == 0:
            print(batch_idx)
            with torch.no_grad():
                pred_df = post_proc(out_sim, ret='df')
                px_size = cfg.evaluation.px_size_zyx
                target_df = sample_to_df(*sim_vars[:5], px_size_zyx=px_size)
                eval_logger(pred_df, target_df, batch_idx, data_str='Sim. ')
                wandb.log({'Sim. Metrics/prob_fac': torch.sigmoid(out_sim['logits']).sum().item()/len(target_df)}, step=batch_idx)
                wandb.log({'Sim. Metrics/n_em_fac': len(pred_df)/len(target_df)}, step=batch_idx)
#                 wandb.log({'Prob hist': wandb.Image(plot_prob_hist(out_sim))}, step=batch_idx)

                if cfg.output.log_figs:
                    sl_fig = sl_plot(x, xsim_noise, nm_to_px(pred_df, px_size), nm_to_px(target_df, px_size), background, out_sim)
                    plt.show()
                    wandb.log({'SL summary': sl_fig}, step=batch_idx)

                if eval_dict is not None:
                    res_eval = model(eval_img[None].cuda())
                    ae_img = microscope(*post_proc(res_eval, ret='micro')[:6])
                    pred_eval_df = post_proc(res_eval, ret='df')
                    wandb.log({'AE Losses/N preds(eval)': len(pred_eval_df)}, step=batch_idx)
                    
                    if eval_df is not None:
                        eval_logger(pred_eval_df, eval_df, batch_idx, data_str='Inp. ')
                        
                    if eval_psf is not None:
#                         wandb.log({'AE Losses/Corr(psf)': cpu(torch.sqrt(torch.mean(((eval_psf-psf.psf_volume.detach().cpu()))**2)))}, step=batch_idx)
                        wandb.log({'AE Losses/Corr(psf)': np.corrcoef(cpu(eval_psf).reshape(-1), cpu(psf.psf_volume).reshape(-1))[0,1]}, step=batch_idx)
                        wandb.log({'AE Losses/RMSE(psf)': np.sqrt(np.mean((cpu(eval_psf/eval_psf.max())-cpu(psf.psf_volume/psf.psf_volume.max()))**2))}, step=batch_idx)
        
                    if cfg.output.log_figs:
                        eval_fig = gt_plot(eval_img, nm_to_px(pred_eval_df, px_size), nm_to_px(eval_df, px_size), px_size,ae_img[0]+res_eval['background'][0], psf)
                        plt.show()
                        wandb.log({'GT': eval_fig}, step=batch_idx)

            # storing
            if batch_idx > 0 and abs(cfg.training.start_psf - batch_idx)<cfg.output.log_interval:
                Path.mkdir(save_dir/'sl_save', exist_ok=True)
                save_train_state(save_dir/'sl_save', model, microscope, optim_net, psf, optim_psf)     
            
            save_train_state(save_dir, model, microscope, optim_net, psf, optim_psf) 
            
    wandb.finish()

In [ ]:
cfg = OmegaConf.load('../config/experiment/covid_early_smFISH_4.yaml')
# cfg.run_name = 'rab11_nb'
# cfg.training.start_micro = 1000
# cfg.training.start_psf = 1000
# cfg.microscope.int_conc = 2.

psf, noise, micro = load_psf_noise_micro(cfg)
post_proc = hydra.utils.instantiate(cfg.post_proc_isi)

In [ ]:
cfg.output.save_dir

'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/covid_early/nb_run/covid_early_smFISH_4'

In [ ]:
cfg.training.start_micro = 0
cfg.training.start_psf = 0

# cfg.data_path.model_init = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/Fig_sim_density/sweep_ff21/int_conc:2.0/sl_save'
cfg.data_path.model_init = cfg.output.save_dir + '/sl_save'

In [ ]:
img_3d, decode_dl = get_dataloader(cfg)

Crop size larger than volume in at least one dimension. Crop size changed to (25, 48, 48)
1 volumes


In [ ]:
# plot_3d_projections(torch.exp(psf.psf_volume[0]))

In [ ]:
# from decode_fish.funcs.visualization import get_simulation_statistics
# get_simulation_statistics(decode_dl, micro, int_threshold=1500)

In [ ]:
inp_offset, inp_scale = get_forward_scaling(img_3d[0])
model = hydra.utils.instantiate(cfg.model, inp_scale=float(inp_scale), inp_offset=float(inp_offset))

psf  .to('cuda')
model.to('cuda')
micro.to('cuda')

Microscope(
  (noise): sCMOS()
)

In [ ]:
opt_net = hydra.utils.instantiate(cfg.training.net.opt, params=model.unet.parameters())
opt_psf = hydra.utils.instantiate(cfg.training.psf.opt, params=list(psf.parameters()))
opt_mic = hydra.utils.instantiate(cfg.training.micro.opt, params=list(model.int_dist.parameters())[:3])

scheduler_net = hydra.utils.instantiate(cfg.training.net.sched, optimizer=opt_net)
scheduler_psf = hydra.utils.instantiate(cfg.training.psf.sched, optimizer=opt_psf)
scheduler_mic = hydra.utils.instantiate(cfg.training.psf.sched, optimizer=opt_mic)

if cfg.evaluation is not None:
    eval_dict = dict(cfg.evaluation)
    eval_dict['crop_sl'] = eval(eval_dict['crop_sl'],{'__builtins__': None},{'s_': np.s_})
    eval_dict['px_size_zyx'] = list(eval_dict['px_size_zyx'])
else:
    eval_dict = None
    
save_dir = Path(cfg.output.save_dir)
save_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
if cfg.data_path.model_init is not None:
    print('loading')
    model = load_model_state(model, cfg.data_path.model_init).cuda()
    micro.load_state_dict(torch.load(Path(cfg.data_path.model_init)/'microscope.pkl'))
    opt_net.load_state_dict(torch.load(Path(cfg.data_path.model_init)/'opt_net.pkl'))
#     opt_psf.load_state_dict(torch.load(Path(cfg.data_path.model_init)/'opt_psf.pkl'))
#     psf.load_state_dict(torch.load(Path(cfg.data_path.model_init)/'psf.pkl'))

loading


In [ ]:
# point_process = PointProcessUniform(local_rate = torch.ones([2,1,48,48,48]).cuda()*.0001, min_int = 0.5, sim_iters=5)
# locs_3d, x_os_3d, y_os_3d, z_os_3d, ints_3d, output_shape = point_process.sample()

In [ ]:
# xsim = micro(locs_3d, x_os_3d, y_os_3d, z_os_3d, ints_3d, output_shape)
# plot_3d_projections(xsim[0,0])

In [ ]:
_ = wandb.init(project=cfg.output.project, 
               config=OmegaConf.to_container(cfg, resolve=True),
               dir=cfg.output.log_dir,
               group=cfg.output.group,
               name=cfg.run_name
          )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aspeiser (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [ ]:
# train(cfg=cfg,
#      model=model, 
#      dl=decode_dl, 
#      optim_net=opt_net, 
#      optim_psf=opt_psf, 
#      optim_mic=opt_mic, 
#      sched_net=scheduler_net, 
#      sched_psf=scheduler_psf, 
#      sched_mic=scheduler_mic, 
#      psf=psf,
#      post_proc=post_proc,
#      microscope=micro, 
#      eval_dict=eval_dict)

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted 17_eval_routines.ipynb.
Converted index.ipynb.
